In [1]:
import requests
import ipywidgets as widgets
import IPython.display
import pandas as pd
import math
import os
import time

In [29]:
url = 'http://localhost:5000/upload'


button = widgets.Button(description="Upload Data")
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
output = widgets.Output()


ui_row = widgets.HBox([button,progress])
dataframe = pd.read_pickle('./ml_data.pkl')

display(dataframe.head(5), ui_row, output)

def upload_chunks(file_path, chunk_length):
    with open(file_path, 'rb') as file:
        start = 0
        file_size = os.path.getsize(file_path)

        headers = {"Filename": os.path.basename(file_path)}
        chunk_count = math.ceil(float(file_size) / chunk_length)
        print("Total chunk count:", chunk_count)
        retry_timeout = 1
        sent_chunk_count = 0
        while True:
            end = min(file_size, start + chunk_length)
            headers['Range'] = "bytes={}-{}/{}".format(start, end, file_size)
            file.seek(start)
            data = file.read(chunk_length)
            start = end
            try:
                response = requests.post(url, headers=headers, data=data)
                if response.ok:
                    print('{}. chunk sent to server'.format(sent_chunk_count + 1))
                    sent_chunk_count += 1
                    progress.value = float(sent_chunk_count)/float(chunk_count)
            except requests.exceptions.RequestException:
                time.sleep(retry_timeout)
                if retry_timeout < 10:
                    retry_timeout += 1
                continue
            if sent_chunk_count >= chunk_count:
                return True
        return False

def on_post_button_clicked(b):
    with output:
        x = upload_chunks('./ml_data.pkl', 4096)
        requests.get('http://localhost:5000/read')

button.on_click(on_post_button_clicked)

,Baseload_baro_clean,Baseload_cit_clean,Baseload_cpd_clean
0,15.042039,38.807115,233.324697
1,15.040953,38.818570,232.953808
2,15.039868,38.830025,232.932536
3,15.038782,38.841480,232.934928
4,15.037697,38.852934,232.618335


Output()